In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install libraries

In [2]:
!pip install transformers -U
!pip install hezar

In [3]:
!pip install pytesseract
!pip install pdf2image
!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-fas

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,502 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [4]:
! pip install --quiet langchain langchain_cohere gpt4all langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [27]:
%cd /content/drive/MyDrive/GAN_class/FirstProject/

/content/drive/MyDrive/GAN_class/FirstProject


## Create agents

In [37]:
import os
from langchain_cohere.chat_models import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain.agents import AgentExecutor
from IPython.display import Audio

from Tool_DataBase import flights_functions, hotel_functions, database_create
from Tool_FAQ import policy_functions
import Tool_Search
from Tool_Search import search_functions
from audio import text_2_speech



os.environ['COHERE_API_KEY'] = "8M3XyFi7BwLJp2wghEbFLpxLySm2sWMwK850DDFO"
llm_model = ChatCohere(model="command-r-plus", temperature=0.3)


# Create the prompt
prompt = ChatPromptTemplate.from_template("{input} Just search in Persian-language in SQLite for every response. Say response only persian language")

my_tools = [search_functions.internet_search,
            policy_functions.lookup_policy,
            flights_functions.search_flights,
            flights_functions.take_ticket,
            flights_functions.cancel_ticket,
            hotel_functions.cancel_hotel,
            hotel_functions.book_hotel,
            hotel_functions.search_hotel]

# Create the ReAct agent
# Create an agent that enables multiple tools to be used in sequence to complete a task.
agent = create_cohere_react_agent(llm = llm_model,
                                  tools = my_tools,
                                  prompt=prompt,)

# Agent that is using tools.
agent_executor = AgentExecutor(agent = agent,
                               tools = my_tools,
                               verbose =False)


# Input is text and output is text too.

In [31]:
questions = {
    "input3":"آیا ساعت، قیمت و دیگر مشخصات پروازها روی سایت علی‌بابا قطعی است؟"
    }

for key, value in questions.items():
    # Invoke the agent with each question
    result = agent_executor.invoke({
        "input": value
    })
    text = result['output']
    print(result['input'], text)

آیا ساعت، قیمت و دیگر مشخصات پروازها روی سایت علی‌بابا قطعی است؟ بله، آنچه پس از جستجوی پرواز مورد نظر خود روی سایت علی‌بابا می‌بینید، برنامه قطعی پروازهای ایرلاین‌هاست که به ریال محاسبه شده است. اگر احیانا تغییری در ساعت یا برنامه‌ی پرواز به وجود بیاید، از طریق پشتیبانی سایت علی‌بابا به اطلاع شما خواهد رسید.


# input is text and output is voice

In [32]:
questions = {
    "input3":"آیا ساعت، قیمت و دیگر مشخصات پروازها روی سایت علی‌ بابا قطعی است؟"
    }

for key, value in questions.items():
    # Invoke the agent with each question
    result = agent_executor.invoke({"input": value},
                                   return_intermediate_steps=False)
    text = result['output']
    audio_file, rate = text_2_speech(text)
    display(Audio(audio_file, rate=rate))

# Input is image and output is text

In [ ]:
img = '/content/Presentation1.jpg'


In [41]:
from hezar.models import Model

model = Model.load("hezarai/crnn-fa-printed-96-long")
texts = model.predict("/content/Presentation1.jpg")
print(f"CRNN Output: {question}")
question = texts[0]['text']
result = agent_executor.invoke({"input": question},
                                   return_intermediate_steps=False)
text = result['output']
print(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CRNN Output: [{'text': 'ه مس مند'}]
متاسفم، متوجه نشدم. لطفا دوباره درخواست خود را مطرح کنید.


# Input is voice and output is text

In [38]:
from hezar.models import Model

model = Model.load("hezarai/whisper-small-fa")

display(Audio('/content/Recording (2).m4a', autoplay=True))

question = model.predict("/content/Recording (2).m4a")
print(question)
result = agent_executor.invoke({"input": question},
                                 return_intermediate_steps=False)
text = result['output']
print(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/hezar/utils/audio_utils.py:34: UserWarning: PySoundFile failed. Trying audioread instead.
  inputs = [librosa.load(x, sr=sampling_rate)[0] for x in paths]
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


[{'text': 'لطفاً مشخصات هتلهای اصفهان را مهم بده.'}]
هتل‌های اصفهان:
- هتل عباسی، خیابان چهارباغ عباسی، ۵,۳۳۳,۰۰۰ تومان
- هتل پارسیان کوثر، خیابان ملت، ۴,۴۰۸,۰۰۰ تومان
- هتل پیروزی، دروازه دولت، ۲,۸۱۹,۰۰۰ تومان
- هتل سفیر، خيابان آمادگاه، ۲,۰۵۰,۰۰۰ تومان
- هتل آپارتمان چهل پنجره، بلوار بوستان سعدی، ۲,۴۴۸,۰۰۰ تومان
- هتل صفوی، چهار راه فلسطين، ۸۶۸,۰۰۰ تومان
- هتل سپاهان، خیابان استانداری، ۱,۵۳۰,۰۰۰ تومان
- هتل آپارتمان پارتیکان، ضلع جنوب غربی میدان نقش جهان، ۷۵۳,۰۰۰ تومان
- هتل باران، خیابان امام خمینی، ۱,۹۰۰,۰۰۰ تومان
- هتل ستاره، میدان امام(نقش جهان)، ۱,۳۵۰,۰۰۰ تومان
- هتل پارسیان سوئیت، ابتدای بلوار آیینه خانه، ۲,۴۰۱,۰۰۰ تومان
- هتل زهره، پل فردوسی، ۱,۲۵۳,۰۰۰ تومان
- هتل سنتی، خیابان سپه، ۲,۳۶۴,۰۰۰ تومان
- هتل دانشمند، خیابان کاوه، ۱,۱۱۳,۰۰۰ تومان
- هتل آزادی، چهار باغ پایین، ۱,۱۶۰,۰۰۰ تومان
- هتل اسپادانا، بلوار آینه خانه، ۱,۹۶۸,۰۰۰ تومان
- هتل شیخ بهایی، چهارباغ عباسی، ۹۷۱,۰۰۰ تومان
